In [1]:
import math

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import pythainlp

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from data_tokenizer import load_corpus

from model.upgrade_sdc import UpgradeSDC
from model.sdc import SDC

### Load Data

In [2]:
file_name = 'ผู้บริโภค - TescoLotus.txt'

corpus, labels = load_corpus('../data/facebook/' + file_name)

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/facebook/tokenized/tokenized_' + file_name)
tokenized_corpus = eval(f.read())
f.close()

Total documents 268
1 clusters


### Preprocess Corpus

#### Remove Words

In [3]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.append('นี้')
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 1449 words
filter frequent words: 605 words
filter letter words: 604 words
filter stop words: 403 words


In [4]:
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [5]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

#### Doc2Vec

In [6]:
tagged_corpus = [TaggedDocument(doc, [i]) for i, doc in enumerate(idx_corpus)]
model = Doc2Vec(tagged_corpus, vector_size=average_doc_size, window=4, min_count=2, epochs=100)
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

paragraph_vectors = [model.infer_vector(doc) for doc in idx_corpus]
paragraph_vectors = pandas.DataFrame(paragraph_vectors, dtype=float)

### Clustering

In [7]:
def get_onehot(corpus, weight):
    dictionary = Dictionary(corpus)
#     dictionary.filter_extremes(no_below=2, no_above=1, keep_n=len(dictionary))

    bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]
    if weight == 'normal':
        weight_corpus = bow_corpus
    elif weight == 'tfidf':
        tfidf = TfidfModel(bow_corpus, smartirs='ltc')
        weight_corpus = [tfidf[doc] for doc in bow_corpus]

    unique_words = [dictionary[id] for id in range(len(dictionary))]
    array = numpy.zeros((len(corpus), len(unique_words)), dtype=float)
    for i, doc in enumerate(weight_corpus):
        for id, score in doc:
            array[i, id] = score

        if weight == 'normal' and len(doc) != 0:
#             array[i] = numpy.divide(array[i], len(idx_corpus[i]))
            array[i] = numpy.divide(array[i], len(doc))
    
    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [8]:
def generate_result(predicted_labels):
    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = filtered_corpus
    result['label'] = labels
    result['predicted_label'] = predicted_labels
    return result

In [9]:
def eval_cluster(onehot_corpus, result):
    label_count = numpy.unique(result['predicted_label'])
    num_cluster = label_count[-1] + 1

    clusters = [[] for i in range(num_cluster)]
    corpus_centroid = []
    for i, label in result['predicted_label'].iteritems():
        clusters[label].append(numpy.array(onehot_corpus.iloc[i]))
        corpus_centroid.append(numpy.array(onehot_corpus.iloc[i]))
    corpus_centroid = numpy.mean(corpus_centroid, axis=0).reshape(1, -1)   

#     print('\tIntra cluster sim\tInter cluster sim\tIntra / Inter')
    compactness = 0
    centroids = []
    for i in range(num_cluster):
        size = len(clusters[i])
        if size != 0:
            centroid = numpy.mean(clusters[i], axis=0)
            centroids.append(centroid)
            centroid = centroid.reshape(1, -1)
            similarities = cosine_similarity(centroid, clusters[i])
            compactness += numpy.sum(similarities)

#             intra = numpy.sum(similarities) / size
#             inter = cosine_similarity(centroid, corpus_centroid)[0][0]
#             print(i, end='\t')
#             print(intra, end='\t')
#             print(inter, end='\t')
#             print(intra / inter)
    return compactness, centroids

In [10]:
min_samples = 7
eps = 0.32
epoch = 10

# onehot_corpus = get_onehot(idx_corpus, 'normal')
onehot_corpus = get_onehot(filtered_corpus, 'tfidf')

max_compactness = 0
for i in range(epoch):
    model = UpgradeSDC()
    _tpredicted_labels, _tmarks = model.predict(onehot_corpus, min_samples, eps)
    
#     _tmarks = None
#     model = DBSCAN(metric='cosine', eps=eps, min_samples=min_samples).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_ + 1

#     model = KMeans(n_clusters=7).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_
    
    _tresult = generate_result(_tpredicted_labels)
    compactness, _tcentroids = eval_cluster(onehot_corpus, _tresult)
    
    if compactness > max_compactness:
        max_compactness = compactness
        predicted_labels = _tpredicted_labels
        marks = _tmarks
        result = _tresult
        centroids = _tcentroids
        
print(max_compactness)
label_count = numpy.unique(result['predicted_label'], return_counts=True) 
num_cluster = label_count[0][-1] + 1
print(label_count, '\n')

107.12412250399923
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]), array([117,   8,  11,   9,  20,  14,  10,   8,   7,  26,   8,  10,  12,
         8])) 



In [11]:
sims = cosine_similarity(centroids)
new_labels = [i for i in range(num_cluster)]
print(new_labels)
for i, row in reversed(list(enumerate(sims))):
    for j, value in reversed(list(enumerate(row[:i + 1]))):
        if i != j and value >= eps - eps / 20:
            print(i, j, value)
            new_labels = [new_labels[j] if label == new_labels[i] else label for label in new_labels]
print(new_labels)

grouped_labels = numpy.zeros(len_corpus)
for i, label in enumerate(predicted_labels):
    grouped_labels[i] = new_labels[label]
new_result = generate_result(grouped_labels)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
13 8 0.4810923160872279
11 4 0.5034346049453596
11 1 0.3414442782023652
11 0 0.4459732482323
10 2 0.48483122344088264
5 4 0.31819192790351114
4 2 0.3704082154205951
4 1 0.5261339400224656
4 0 0.4531235340053271
1 0 0.3371756929043165
[2, 2, 2, 3, 2, 2, 6, 7, 8, 9, 2, 2, 12, 8]


### Result

In [12]:
class Widget:
    def __init__(self, result, marks, column_name):
        self.result = result
        self.column_name = column_name
        self.marks = marks
        
        label_count = numpy.unique(result['predicted_label'])
        self.widget = widgets.ToggleButtons(
            options=[int(num) for num in label_count],
            disabled=False,
            button_style='',
        )
        
        self.widget.observe(self.on_click, names='index')
        self.on_click({'new' : 0})
        
    def on_click(self, change):
        clear_output()
        display(self.widget)
        new = self.widget.options[change['new']]
        for index, value in self.result[self.result['predicted_label'] == new][self.column_name].iteritems():
            if self.marks:
                if index in self.marks[0]:
                    print("@", end="")
                elif index in self.marks[1]:
                    print("*", end="")
            print(index, value)

In [13]:
w1 = Widget(new_result, marks, 'comment')

ToggleButtons(options=(2, 3, 6, 7, 8, 9, 12), value=2)

*0 โลตัสไฮเปอร์มาเก็ต​ ทุกสาขา​ ควรจ้างพนักงานเยอะกว่านี้​ ไม่ควรเรียกพนักงานแผนกอื่นไปลงเครื่องแคชเชียร์​บ่อยจนเกินไป​ ทุกคนต้องมีหน้าที่ที่ตัวเองต้องรับผิดชอบ​ พอช่วยแคชเชียร์เสร็จ​ งานแผนกตัวเองเละก็โดนด่า​ พอโดนด่าก็เกิดความเครียด​ หน้าบึ้งตึงใส่ลูกค้า​ ทำงานไม่มีความสุข​ บริการลูกค้าไม่ดี​ แล้วลูกค้าก็ร้องเรียน​ เพราะรักจึงอยากแนะนำ​ จากใจเด็กโลตัสเก่า​
*1 บางครั้งพนักงานโลตัสเอ็กเพลส กับพนักงานโลตัสใหญ่ ก็ต่างกันครับ โลตัสใหญ่ เขาทำเป็นเเผนกๆไปครับ เเต่โลตัสเอ็กเพลสนั้น พนักงานเเต่ละคน ต้องทำทุกอย่างครับ ไม่ใช่เเค่ยืนขายของ อยากให้ทุกคนเข้าใจด้วยครับ บางคนเวลากินข้าวเเทบไม่มี ผมเชื่อว่าทุกคนก็เป็นพนักงานเงินเดือนเหมือนกัน ควรเข้าใจกันบ้างครับ พนักงานคนอื่นเป็นเเบบไหนผมไม่รู้ เเต่มันก็ยังมีพนักงานที่ตั้งใจทำงานครับ ขอให้ทุกคนเข้าใจครับ
2 เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพียว ได้แล้ว!!!!
4 ปรับปรุงพนักงานโลตัสเอ็กซ์เพลสสาขาตลาดนานาเจริญ ห่วยแตก ไม่เต็มใจทำก็ออกไปทำงานอย่างอื่น คนอื่นที่เข้าอยากทำงานจะได้เข้ามา ออกไป๊
*7 ..เคยไปทำ เอ้กซ์เพรส เดือนเดียว พอเลย  พนักงานต่อ1ช่

*65 พนักงานอย่างเดียวเลย ไปขอใบกำกับภาษี ทำหน้าแบบผัวไม่เยมสชาติกว่า พูดก็ไม่ดี อยากเอายางไปดีดหีเรียงตัว มึงทำงานบริการ มึงมีสิทธิ์อะไรมางี่เง่าใส่ลูกค้าอ่ะ บ้าบอมาก
66 เกือบทุกสาขา เวลามีโปรโมชั่นอะไร ไม่รู้ มาถามลูกค้า ว่าลดมั้ย ลดด้วยเหรอ เอิ่ม... แล้วตูจะรู้มั้ยอ่ะ บางทีก็งงๆ กะพนักงาน
67 โลตัสเอ็กเพรสแย่หน้ายังกะผัวทิ้ง. น่าจับอบรมให้ดีกว่านี้
69 หมูสับในโลตัสเอ็กเพรสบางไปค่ะ บดซะแห้งเกิน ผักช้ำๆ บางอันก็เสียแล้วจริงๆ ทิ้งเถอะค่ะ อย่าป้ายเหลืองเลย
70 โลตัสเอ็กเพลส ไม่สามารถจ่ายบิลค่าไฟได้พร้อมกันได้จ้า ต้องแยกจ่าย
74 สันดานพนักงาน โดยเฉพาะ express ควรปรับปรุงอย่างรุนแรง ขอโทษนะลูกค้าไม่ได้ไปแบมือขอฟรี
75 เวลาผมซื้อเนื้อสัตว์ช่วยเอาที่คีบหรือช้อนหรืออะไรก็ได้มาวางให้ทีอย่าให้กุต้องใช้มือเปล่าๆจับอีกอย่างจะเก็บช้อนกะที่คีบเร็วไปไหนเพิ่งจะ6โมงเย็นเห็นเอาไปล้างแล้ว
76 พนักงานสาขาย่อย เขาเรียกเอกซ์เพรสปะ หน้าบูดกวนตีน ถามอะไรไม่ตอบ เป็นหบายสาขาเลย เคยยืนรอให้มันคิดตังให้ ต้องเรียกอะ ขนาดวางของที่เค้าเตอร์แล้ว มัวยืนคุยกับเพื่อน ทั้งๆที่มันก็เห็น
*77 พนักงานหน้าเคาเตแจุดบริการลูกค้า​ ห

In [14]:
w2 = Widget(result, marks, 'comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13), value=0)

2 เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพียว ได้แล้ว!!!!
8 อย่าบังคับน้องเข้าประชุมเชียร์ อย่าลงโทษโดยเหตุผลงี่เง่าๆ เกิดก่อนไม่กี่ปีเอง
9 โลตัสใหญ่ไม่มีอะไรนะ โลตัสเอกเพรสแม่งไม่ไหวจริงๆ เกลียดขี้หน้าพนักงาน
14 เลือกผัก เลือกแล้ว เลือกอีก พอมาคิดเงินพนักงานยัดผักใส่ถุงจนผักหัก โอ้ยใจ ใจสลาย
15 มารยาทพนักงาน เซเว่นยังคุณภาพดีกว่าหน่อยนึง แบบว่าเหี้ยทั้งคู่เลย
19 น่าเห็นใจนะฮะ พนักงานโดนสวดยับตั้งแต่เช้า พอหน้าบูด ลูกค้าก็ไม่ชอบ
23 เช็คเอาท์ ... หน้าจะงอไปไหนสวัสดีค่ะมีอะไรให้ช่วยมั้ยคะโอกาสหน้าเชิญใหม่ค่ะมีคลับการ์ดมั้ยคะแค่เนี้ยะ ... จะตายมั้ย
28 Express ทุกสาขา!!! พนักงานหายไปไหน?? ต้องเดินตามหา พอเจอแลเวเรียกมาคิดเงิน ก็อารมณ์เสียหน้าบูด... สงสัยลูกค้าจะรบกวนเค้ามั้ง??? ของน่ะ จะขายมั้ย???
29 โลตลาด พนง.ปากตลาด นินทาลค.เผาขน ตะโกนโหวกเหวกข้ามหัวลค. วันหวยออกสนั่นเป็นพิเศษ เห็นพฤติกรรมแล้วแย่ ปรับปรุงภาพลักษณ์เถอะ
30 ไม่อยากจะเม้าความสาระพัดของโลตัส ในฐานะที่เคยเป็นพนง.มาก่อน อิอิ
32 ไม่ต้องตัดแต้มเป็นคูปองเงินได้ไหมอ่ะ ไม่เคยได้ใช้เพราะ คูปองมันหายก่อน เอาแบบ พอไปซื้อแล้วแต้มคร

In [15]:
w3 = Widget(result, marks, 'tokenized_comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13), value=0)

2 ['ดี', 'เลิก', 'เครื่อง', 'น้ำ', 'ข้าว', 'อี']
8 ['อย่า', 'อย่า', 'กี่', 'ปี', 'บังคับ', 'น้อง', 'เชียร์', 'งี่', 'เง่า']
9 ['พนักงาน', 'โลตัส', 'โลตัส', 'แม่ง', 'เกลียด', 'เอกเพรส']
14 ['พนักงาน', 'เงิน', 'ถุง', 'ใส่', 'ผัก', 'ผัก', 'ผัก', 'เลือก', 'เลือก', 'เลือก', 'ยัด']
15 ['พนักงาน', 'ดี', 'มารยาท', 'นึง', 'เซเว่น', 'คุณภาพ', 'เหี้ย', 'คู่']
19 ['พนักงาน', 'ลูกค้า', 'หน้า', 'โดน', 'ชอบ', 'บูด', 'เช้า', 'เห็นใจ', 'ฮะ']
23 ['หน้า', 'หน้า', 'การ์ด', 'คลับ', 'เช็ค', 'งอ', 'สวัสดี', 'ตาย', 'โอกาส']
28 ['พนักงาน', 'สาขา', 'ลูกค้า', 'หน้า', 'เงิน', 'เดิน', 'เจอ', 'ขาย', 'หา', 'บูด', 'หาย']
29 ['ปรับปรุง', 'พนง', 'แย่', 'ตะโกน', 'ข้าม', 'หวย', 'พิเศษ', 'พฤติกรรม']
30 ['โลตัส', 'พนง', 'ฐานะ']
32 ['เงิน', 'ซื้อ', 'ดี', 'ถาม', 'อ่ะ', 'หาย', 'เล่น', 'ไหม', 'ไหม', 'คู', 'ปอง']
34 ['เงิน', 'ราคา', 'ตอน', 'เต็ม', 'บาท', 'สีหน้า', 'เศษ', 'สตางค์', 'เศษ', 'ชัก']
35 ['ลูกค้า', 'จ่าย', 'พนง', 'พนง', 'ติด', 'ใบ', 'ใบ', 'ใบ', 'ดู', 'ดู', 'นึง']
37 ['สาขา', 'ทำ', 'ทำ', 'ทำ', 'ปรับปรุง', 'เรื่อง', 'ถา

In [63]:
seed = 4
compare = 19

a = numpy.array(onehot_corpus.iloc[seed]).reshape(1, -1)
b = numpy.array(onehot_corpus.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))
print(filtered_corpus[seed])
print(filtered_corpus[compare])

# print(sims[10])

[[0.00923844]]
['พนักงาน', 'สาขา', 'ทำ', 'ทำ', 'ทำ', 'คน', 'งาน', 'งาน', 'ปรับปรุง', 'เต็มใจ', 'โลตัสเอ็กซ์']
['พนักงาน', 'ลูกค้า', 'หน้า', 'โดน', 'ชอบ', 'บูด', 'เช้า', 'เห็นใจ', 'ฮะ']
